## Forfatter(e)
1. Magne Furuholmen Myhren (høsten 2022). Basert på en rekke sas-filer, tidligere drop-lister og spesifikk kompetanse fra kollegaer

# Setup

## Velger årgang og delregister
Legger også til filsti til rådata fra NPR og avtalsespesialister

In [124]:
aargang = 2022
siste_to_siffer_aargang = str(aargang)[-2:]

aar_for = aargang - 1                          # året før
siste_to_siffer_aar_for = str(aar_for)[-2:]

lagre_filer = False                            # sett til True, hvis du vil lagre droplistene
sti_til_lagring = ""                           # eventell sti til lagringsmappe. 
                                               # Lagres i denne mappen hvis tom

## Importerer pakker og setter innstillinger

In [125]:
import numpy as np
import pandas as pd
import cx_Oracle
import getpass
import os
import matplotlib.pyplot as plt
import requests
from pandas.io.json import json_normalize #package for flattening json in pandas df

# Fjerner begrensning på antall rader og kolonner som vises av gangen
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

# Unngå standardform i output
pd.set_option('display.float_format', lambda x: '%.0f' % x)

## Tilgang oracle

In [126]:
conn = cx_Oracle.connect(getpass.getuser()+"/"+getpass.getpass(prompt='Oracle-passord: ')+"@DB1P")

Oracle-passord:  ···········


# Henter data

##  fra KLASS

### Offentlige helseforetak

In [127]:
## Klass for å finne skjemanavn i dynarev
# URL = f'http://data.ssb.no/api/klass/v1/classifications/610/codes.json?from={aargang}-01-01&includeFuture=True'
# r = requests.get(url = URL)
# skjemanavn_klass = pd.read_json(r.text)
# skjemanavn_klass = pd.json_normalize(skjemanavn_klass['codes'])

# skjemanavnSOM = skjemanavn_klass.query('level=="2" and parentCode == "SOM"')[['code','name', 'parentCode']]
# skjemanavnSOM

In [128]:
URL = f'http://data.ssb.no/api/klass/v1/classifications/603/codes.json?from={aargang}-01-01&includeFuture=True'
r = requests.get(url = URL)
offhelse_df = pd.read_json(r.text)
offhelse_df = pd.json_normalize(offhelse_df['codes'])

RHF_kode_klass = offhelse_df.query('level=="1"')[['code','name']]
RHF_kode_klass = RHF_kode_klass.rename(columns = {'code':'HELSEREGION', 'name': 'RHF'})

lvl2 = offhelse_df.query("level == '2'")
RHF_kode_klass = pd.merge(RHF_kode_klass, lvl2, how="left", left_on="HELSEREGION", right_on="parentCode")
RHF_kode_klass = RHF_kode_klass.rename(columns = {'code': 'ORGNR_FORETAK'})
RHF_kode_klass = RHF_kode_klass[['HELSEREGION', 'RHF', 'ORGNR_FORETAK']]

lvl2 = offhelse_df.query("level == '2'")
lvl3 = offhelse_df.query("level == '3'")
lvl2 = lvl2[['code', 'parentCode', 'name']]
lvl3 = lvl3[['code', 'parentCode', 'name']]

lvl2 = lvl2.rename(columns = {'parentCode':'HELSEREGION', 'name': 'RHF'})
lvl3 = lvl3.rename(columns = {'code':'ORGNR_FORETAK', 'name': 'NAVN_KLASS'})

In [129]:
HF = pd.merge(lvl3, lvl2, how="left", left_on="parentCode", right_on="code")
HF = HF[['ORGNR_FORETAK', 'NAVN_KLASS', 'HELSEREGION', 'RHF']]

RHF_kode_klass['RHF'] = RHF_kode_klass['RHF'] + " RHF"
RHF_kode_klass['NAVN_KLASS'] = RHF_kode_klass['RHF']

### Private helseinstitusjoner med oppdrags- og bestillerdokument

In [130]:
URL = f'http://data.ssb.no/api/klass/v1/classifications/604/codes.json?from={aargang}-01-01&includeFuture=True'
r = requests.get(url = URL)
privhelse_df = pd.read_json(r.text)
privhelse_df = pd.json_normalize(privhelse_df['codes'])

temp = privhelse_df.query("level == '2'")
temp = temp[['code', 'parentCode', 'name']]
temp = temp.rename(columns = {'code':'ORGNR_FORETAK','parentCode':'HELSEREGION', 'name': 'NAVN_KLASS'})
temp = pd.merge(temp, RHF_kode_klass, how="left")

rapporteringsenheter = pd.concat([HF,temp])
rapporteringsenheter = pd.concat([rapporteringsenheter,RHF_kode_klass])

### Regionale og felleseide støtteforetak i spesialisthelsetjenesten

In [131]:
URL = f'http://data.ssb.no/api/klass/v1/classifications/605/codes.json?from={aargang}-01-01&includeFuture=True'
r = requests.get(url = URL)
regfel_df = pd.read_json(r.text)
regfel_df = pd.json_normalize(regfel_df['codes'])

lvl2o = regfel_df.query("level == '2' & parentCode != '99'")
lvl3o = regfel_df.query("level == '3'")
lvl2o = lvl2o[['code', 'parentCode', 'name']]
lvl3o = lvl3o[['code', 'parentCode', 'name']]
lvl2o = lvl2o.rename(columns = {'parentCode':'HELSEREGION',
                                'name': 'RHF'})
lvl3o = lvl3o.rename(columns = {'code': 'ORGNR_FORETAK',
                                'name': 'NAVN_KLASS'})

temp = pd.merge(lvl3o, lvl2o, how="left", left_on="parentCode", right_on="code")
temp = temp[['ORGNR_FORETAK', 'NAVN_KLASS', 'HELSEREGION', 'RHF']]

rapporteringsenheter = pd.concat([rapporteringsenheter,temp])

temp = regfel_df.query("level == '2' & parentCode == '99'")
temp = temp.rename(columns = {'code':'ORGNR_FORETAK', 'name': 'NAVN_KLASS', 'parentCode': 'HELSEREGION'})
temp['RHF'] = "FELLESEIDE STØTTEFORETAK"
temp = temp[['ORGNR_FORETAK', 'NAVN_KLASS', 'HELSEREGION', 'RHF']]

In [132]:
rapporteringsenheter = pd.concat([rapporteringsenheter,temp])

### Limer sammen all data fra KLASS

In [133]:
alle_foretak_virk = pd.concat([offhelse_df, privhelse_df, regfel_df])
alle_foretak_virk = alle_foretak_virk[["code", "parentCode", "level", "name"]]
alle_foretak_virk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 0 to 23
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   code        66 non-null     object
 1   parentCode  55 non-null     object
 2   level       66 non-null     object
 3   name        66 non-null     object
dtypes: object(4)
memory usage: 2.6+ KB


## VOF: liste over alle helseforetak

In [134]:
rapporteringsenheter_uten_RHF = rapporteringsenheter.query('~NAVN_KLASS.str.endswith("RHF")',engine="python")

In [135]:
r_orgnr = rapporteringsenheter_uten_RHF.ORGNR_FORETAK.to_numpy()

In [136]:
def lag_sql_str(arr):
    s = "("
    for nr in arr:
        s += "'" + str(nr) + "',"
    s = s[:-1] + ")"
    return s

sql_str = lag_sql_str(r_orgnr)

sporring_for = f"""
    SELECT FORETAKS_NR, ORGNR, NAVN
    FROM DSBBASE.SSB_FORETAK
    WHERE STATUSKODE = 'B' AND ORGNR IN {sql_str}
"""
vof_for = pd.read_sql_query(sporring_for, conn)

fornummer = pd.Series(vof_for['FORETAKS_NR']).array
sql_str = lag_sql_str(fornummer)

sporring_bed = f"""
    SELECT FORETAKS_NR, ORGNR, NAVN, KARAKTERISTIKK, SN07_1, SB_TYPE
    FROM DSBBASE.SSB_BEDRIFT
    WHERE STATUSKODE = 'B' AND FORETAKS_NR IN {sql_str}
"""
vof_bdr = pd.read_sql_query(sporring_bed, conn)



In [137]:
# Henter organisasjons- og foretaksnummer fra Virksomhets- og 
# foretaksregisteret (VoF) og samler disse i én tabell kalt vof

vof_for = vof_for.rename(columns = {'NAVN':'NAVN_FORETAK'})
vof_for = vof_for.rename(columns = {'ORGNR':'ORGNR_FORETAK'})

vof_bdr = vof_bdr.rename(columns = {'ORGNR':'ORGNR_BEDRIFT'})
vof_bdr['KARAKTERISTIKK'] = vof_bdr['KARAKTERISTIKK'].fillna("")
vof_bdr['NAVN_BEDRIFT'] = vof_bdr['NAVN'] + " " + vof_bdr['KARAKTERISTIKK']

vof_bdr=vof_bdr.drop(columns=['NAVN', 'KARAKTERISTIKK'])

vof = pd.merge(vof_bdr,vof_for, how='left', on='FORETAKS_NR')
vof = vof.drop(columns=['FORETAKS_NR'])

rapporteringsenheter['ORGNR_FORETAK'] = rapporteringsenheter['ORGNR_FORETAK'].apply(str)
rapporteringsenheter_vof = pd.merge(vof,rapporteringsenheter, how='left', on='ORGNR_FORETAK')

In [138]:
rapporteringsenheter_vof.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1237 entries, 0 to 1236
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ORGNR_BEDRIFT  1237 non-null   object
 1   SN07_1         1237 non-null   object
 2   SB_TYPE        1237 non-null   object
 3   NAVN_BEDRIFT   1237 non-null   object
 4   ORGNR_FORETAK  1237 non-null   object
 5   NAVN_FORETAK   1237 non-null   object
 6   NAVN_KLASS     1237 non-null   object
 7   HELSEREGION    1237 non-null   object
 8   RHF            1202 non-null   object
dtypes: object(9)
memory usage: 96.6+ KB


## Dynarev

Henter ut SFU og tar senere vare på de som har har non-missing på ```SKJEMA_TYPE```.

In [139]:
sporring = f"""
    SELECT *
    FROM DSBBASE.DLR_ENHET_I_DELREG
    WHERE DELREG_NR IN ('24{siste_to_siffer_aargang}')
"""
SFU_data = pd.read_sql_query(sporring, conn)
print(f"Rader:    {SFU_data.shape[0]}\nKolonner: {SFU_data.shape[1]}")
SFU_data.info()


Rader:    1591
Kolonner: 127
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591 entries, 0 to 1590
Columns: 127 entries, DELREG_NR to STR_KODE
dtypes: datetime64[ns](4), float64(2), int64(1), object(120)
memory usage: 1.5+ MB


In [140]:
def hent_data_delreg24x_og_19377x(x, skjema):
    sporring = f"""
        SELECT *
        FROM DYNAREV.VW_SKJEMA_DATA
        WHERE DELREG_NR IN ('24{x}', '19377{x}') AND SKJEMA IN ('{skjema}') AND AKTIV = '1'
    """
    sporring_df = pd.read_sql_query(sporring, conn)
    sporring_df = sporring_df[['SKJEMA','ENHETS_ID', 'FELT_ID', 'FELT_VERDI']]
    sporring_df = sporring_df.pivot(index=['ENHETS_ID','SKJEMA'], columns='FELT_ID', values='FELT_VERDI')
    return sporring_df

In [141]:
skjemadata38O = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE38O')
skjemadata38P = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE38P')
skjemadata39  = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE39')
skjemadata40  = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE40')
skjemadata41  = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE41')
skjemadata44O = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE44O')
skjemadata44P = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE44P')
skjemadata45O = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE45O')
skjemadata45P = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE45P')
skjemadata46O = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, '46O')           # skj46 bytter navn for ulike aarganger (fiks med klass?)
skjemadata46P = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE46P')
skjemadata47  = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE47')
skjemadata48  = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE48')

# Setter sammen master-SFU-fil med all nødvendig informasjon

SFU: orgnummer, orgnummer_foretak, foretaksnavn, helseregionnavn, helseregionsnummer
"alle virksomhetene til speshelse ligger i 2421"


In [142]:
SFUklass = pd.merge(SFU_data, rapporteringsenheter, how="left", on="ORGNR_FORETAK")

SFUklass['HELSEREGION'] = SFUklass['HELSEREGION'].fillna("06")

SFUklass['SKJEMA_TYPE'] = SFUklass['SKJEMA_TYPE'].apply(lambda x: str(x).split(" "))
SFUklass[['NAVN1', 'NAVN2', 'NAVN3', 'NAVN4', 'NAVN5']] = SFUklass[['NAVN1', 'NAVN2', 'NAVN3', 'NAVN4', 'NAVN5']].fillna("")

In [143]:
# Slår sammen alle navnekolonnene til en adskilt med ett mellomrom
SFUklass['NAVN'] = SFUklass['NAVN1'] + " " +\
                        SFUklass['NAVN2'] + " " +\
                        SFUklass['NAVN3'] + " " +\
                        SFUklass['NAVN4'] + " " +\
                        SFUklass['NAVN5']
SFUklass['NAVN'] = SFUklass['NAVN'].apply(lambda x: ' '.join(x.split()))

# Lager droplister
Sortert etter hva slags dropliste som skal lages.

In [144]:
skjemaer_til_droplister = ["skj41", "skj0X", "skj0Y", "skj38O", "skj38P", "skj39", "skj40",
                           "skj44O", "skj44P", "skj45O", "skj45P", "skj46O",
                           "skj46P", "skj47", "skj48"]

# kolvis er spesielle kolonner som tas ut av en større tabell
kolvis = ['NAVN1', "NAVN", "ORGNR", "ORGNR_FORETAK", "HELSEREGION", "RHF"]

In [145]:
# nyttig funksjon som returnerer en tabell med alle rader som inneholder et gitt skjema
# eks: 
# tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "0X")

def tabell_som_inneholder_skjema(df, kol, skjemanavn):
    ant = df.shape[0]
    rader_med_skjemanavn = []
    for i in range(ant):
        if skjemanavn in df[kol].iloc[i]:
            rader_med_skjemanavn.append(i)
    return df.iloc[rader_med_skjemanavn]


## Skjemaer som hentes rett fra klass og SFU

### 0X (Resultatregnskap for RHF og HF)

In [146]:
# kolonner som skal være med i dropdownlisten:
kolonner0X0Y404148 = ["USERID", "REGION_NR", "REGION_NAVN", "FORETAK_ORGNR", "FORETAK_NAVN"]

skj0X = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "0X")[kolvis]
skj0X = skj0X.rename(columns = {'NAVN1':'FORETAK_NAVN',
                                'ORGNR':'USERID',
                                'HELSEREGION':'REGION_NR',
                                'RHF':'REGION_NAVN'})
skj0X['FORETAK_ORGNR'] = skj0X['USERID']

skj0X = skj0X[kolonner0X0Y404148].sort_values('REGION_NR')

### 0Y (Balanseregnskap for RHF og HF)

In [147]:
skj0Y = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "0Y")[kolvis]
skj0Y = skj0Y.rename(columns = {'NAVN1':'FORETAK_NAVN', 
                                'ORGNR':'USERID', 
                                'HELSEREGION':'REGION_NR', 
                                'RHF':'REGION_NAVN'})
skj0Y['FORETAK_ORGNR'] = skj0Y['USERID']

skj0Y = skj0Y[kolonner0X0Y404148].sort_values('REGION_NR')

### skj40 (Kontantstrømoppstilling)

In [148]:
skj40 = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "0Y")[kolvis]
skj40 = skj40.rename(columns = {'NAVN1':'FORETAK_NAVN',
                                'ORGNR':'USERID',
                                'HELSEREGION':'REGION_NR',
                                'RHF':'REGION_NAVN'})
skj40['FORETAK_ORGNR'] = skj40['USERID']

skj40 = skj40[kolonner0X0Y404148].sort_values('REGION_NR')

### skj41 (Private spesialister med driftsavtale)

In [149]:
skj41 = RHF_kode_klass.copy()
skj41 = skj41.rename(columns = {'HELSEREGION': 'REGION_NR',
                                'RHF': 'REGION_NAVN',
                                'ORGNR_FORETAK': 'FORETAK_ORGNR'})
skj41['USERID'] = skj41['FORETAK_ORGNR']
skj41['FORETAK_NAVN'] = skj41['REGION_NAVN']
skj41 = skj41[kolonner0X0Y404148]

### skj48 (Praksiskonsulentordningen)

In [150]:
skj48 = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "48").copy()
skj48 = skj48[["NAVN_KLASS", "ORGNR_FORETAK", "HELSEREGION"]]
skj48 = pd.merge(skj48, RHF_kode_klass[['HELSEREGION', 'RHF']], how="left", on="HELSEREGION")
skj48 = pd.concat([skj48, HF])
skj48 = skj48.rename(columns={"NAVN_KLASS": "FORETAK_NAVN",
                              "ORGNR_FORETAK": "FORETAK_ORGNR",
                              "HELSEREGION": "REGION_NR",
                              "RHF": "REGION_NAVN"})
skj48['USERID'] = skj48['FORETAK_ORGNR']

skj48 = skj48[kolonner0X0Y404148] \
    .sort_values("REGION_NR") \
    .reset_index(drop=True)

## Skjemaer til offentlige foretak
Inneholder et visst antall kolonner med virksomheters navn og orgnr.

In [151]:
def lag_navn_orgnr_kolonnenavn(ant_kolonner):
    nvn, org = [], []
    for i in range(1,ant_kolonner+1):
        nvn.append(f'NAVN_VIRK{i}')
        org.append(f'ORGNR_VIRK{i}')
    return nvn, org

# ------------------------------------------------------------------------------------

# Denne funksjonen tar inn en dataframe med undervirksomheter som hører til et foretaksnummer og antall kolonner som skal 
# være med i droplisten. Undervirksomhetene legges etterhverandre med riktig korrespondanse mellom navn- og orgnr-kolonner.
def lag_navn_orgnr_kolonner(fvdf, ant_kolonner, med_foretak = True):
    foretak_rader = fvdf.ORGNR_FORETAK.value_counts().index.to_numpy()              # tar ut unike foretaksnummer i en liste

    foretak_org_df = -1
    for nr in foretak_rader:                                                        # blar gjennom listen og lager kolonner til hvert foretak
        if med_foretak:
            orgnr_df = fvdf.query(f'ORGNR_FORETAK  == "{nr}"')['ORGNR']
            navn_df = fvdf.query(f'ORGNR_FORETAK  == "{nr}"')['NAVN']
        else:
            orgnr_df = fvdf.query(f'ORGNR_FORETAK  == "{nr}" and ORGNR != "{nr}"')['ORGNR']
            navn_df = fvdf.query(f'ORGNR_FORETAK  == "{nr}" and ORGNR != "{nr}"')['NAVN']    

        ant_virksomheter = orgnr_df.shape[0]
        tom_df = pd.Series(np.nan, index=range(ant_kolonner-ant_virksomheter))
        orgnr_df = pd.concat([orgnr_df, tom_df], axis=0)
        orgnr_df.index = orgnr_virk

        ant_navn = navn_df.shape[0]
        tom_df = pd.Series(np.nan, index=range(ant_kolonner-ant_navn))
        navn_df = pd.concat([navn_df, tom_df], axis=0)
        navn_df.index = navn_virk

        foretak_df = pd.concat([orgnr_df, navn_df], axis=0)
        foretak_df['ORGNR_FORETAK'] = nr

        if type(foretak_org_df) == int:                                             # limer sammen hvert foretak med nye kolonner til en stor tabell
            foretak_org_df = foretak_df
        else:
            foretak_org_df = pd.concat([foretak_org_df, foretak_df], axis = 1)
    return foretak_org_df.transpose()

### skj38O (TSB for offentlige helseforetak)

Alle offentlige helseforetak foruten ```SUNNAAS```. Bruker ```HF```-tabellen laget i kapittel 2.1.1

OBS: Nye enheter 2022

In [152]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata38O[['FORETAKETS_ORGNR','DGN_DGN']].reset_index().copy()
                [['FORETAKETS_ORGNR','DGN_DGN']]
                .rename(columns={'DGN_DGN': 'D_PLAS_FJOR'}))

In [153]:
skj38O = HF.query('ORGNR_FORETAK != "883971752"').copy()                           # Tar ut SUNNAAS

In [154]:
# Lager kolonneoverskrifter i tråd med tidligere droplister:
navn_virk, orgnr_virk = lag_navn_orgnr_kolonnenavn(20)

onskede_kolonner = ["USERID", "HELSEREGION", "HELSEREGION_NAVN",
                    "FORETAKETS_ORGNR", "FORETAKETS_NAVN",
                    "D_PLAS_FJOR"] + orgnr_virk + navn_virk

Henter data fra SFU med næringskode ("86.106") og statuskode ("B")
Næringskode i kolonne SN07_1

In [155]:
skj38O['tmp_bool'] = True

In [156]:
finne_virksomheter_df = pd.merge(SFUklass, skj38O, how="left", on=["ORGNR_FORETAK", "RHF", "NAVN_KLASS", "HELSEREGION"])
finne_virksomheter_df = finne_virksomheter_df.query('tmp_bool == True and SN07_1 == "86.106" and STATUS == "B"')
finne_virksomheter_df = finne_virksomheter_df[['ORGNR','ORGNR_FORETAK','NAVN']]

In [157]:
undervirksomheter_navn_og_kolonner = lag_navn_orgnr_kolonner(finne_virksomheter_df, 20)

In [158]:
skj38O = pd.merge(skj38O, undervirksomheter_navn_og_kolonner, how="left", on="ORGNR_FORETAK")

skj38O['USERID'] = skj38O['ORGNR_FORETAK']
skj38O = skj38O.rename(columns={"ORGNR_FORETAK": "FORETAKETS_ORGNR",
                                "NAVN_KLASS": "FORETAKETS_NAVN",
                                "RHF": "HELSEREGION_NAVN"})

skj38O = pd.merge(skj38O, d_plass_fjor, how="left", on="FORETAKETS_ORGNR")

skj38O = skj38O[onskede_kolonner]

### skj44O (Psykisk helsevern for voksne PHFV)

OBS: Nye enheter 2022

Alle offentlige helseforetak foruten ```SUNNAAS```. Bruker ```HF```-tabellen laget i kapittel 2.1.1

In [159]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata44O[['FORETAKETS_ORGNR','D_PLAS_T']].reset_index().copy()
                [['FORETAKETS_ORGNR','D_PLAS_T']]
                .rename(columns={'D_PLAS_T': 'D_PLAS_FJOR'}))

In [160]:
skj44O = HF.query('ORGNR_FORETAK != "883971752"').copy()                    # Tar ut SUNNAAS

In [161]:
# Lager kolonneoverskrifter i tråd med tidligere .csv-filer:
navn_virk, orgnr_virk = lag_navn_orgnr_kolonnenavn(20)

onskede_kolonner = ["USERID", "HELSEREGION", "HELSEREGION_NAVN",
                    "FORETAKETS_ORGNR", "FORETAKETS_NAVN",
                    "D_PLAS_FJOR"] + orgnr_virk + navn_virk

Henter data fra SFU med næringskode ("86.104") og statuskode ("B"). Finner næringskoden i kolonne ```SN07_1```

In [162]:
skj44O['tmp_bool'] = True

In [163]:
finne_virksomheter_df = pd.merge(SFUklass, skj44O, how="left", on=["ORGNR_FORETAK", "RHF", "NAVN_KLASS", "HELSEREGION"])
finne_virksomheter_df = finne_virksomheter_df.query('tmp_bool == True and SN07_1 == "86.104" and STATUS == "B"')
finne_virksomheter_df = finne_virksomheter_df[['ORGNR','ORGNR_FORETAK','NAVN']]

In [164]:
undervirksomheter_navn_og_kolonner = lag_navn_orgnr_kolonner(finne_virksomheter_df, 20)

In [165]:
skj44O = pd.merge(skj44O, undervirksomheter_navn_og_kolonner, how="left", on="ORGNR_FORETAK")

skj44O['USERID'] = skj44O['ORGNR_FORETAK']
skj44O['USERID'] = skj44O['ORGNR_FORETAK']
skj44O = skj44O.rename(columns={"ORGNR_FORETAK": "FORETAKETS_ORGNR",
                                "NAVN_KLASS": "FORETAKETS_NAVN",
                                "RHF": "HELSEREGION_NAVN"})

skj44O = pd.merge(skj44O, d_plass_fjor, how="left", on="FORETAKETS_ORGNR")

skj44O = skj44O[onskede_kolonner]

### skj45O (Psykisk helsevern for barn og unge (PHBU/BUP), offentlige helseforetak)

In [166]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata45O[['FORETAKETS_ORGNR','D_PLAS_T']].reset_index().copy()
                [['FORETAKETS_ORGNR','D_PLAS_T']]
                .rename(columns={'D_PLAS_T': 'D_PLAS_FJOR'}))

In [167]:
skj45O = HF.query('ORGNR_FORETAK != "883971752"').copy()                    # Tar ut SUNNAAS

In [168]:
# Lager kolonneoverskrifter i tråd med tidligere .csv-filer:
navn_virk, orgnr_virk = lag_navn_orgnr_kolonnenavn(20)

onskede_kolonner = ["USERID", "HELSEREGION", "HELSEREGION_NAVN",
                    "FORETAKETS_ORGNR", "FORETAKETS_NAVN",
                    "D_PLAS_FJOR"] + orgnr_virk + navn_virk

Henter data fra SFU med næringskode ("86.105") og statuskode ("B"). Finner næringskoden i kolonne ```SN07_1```

In [169]:
skj45O['tmp_bool'] = True

In [170]:
finne_virksomheter_df = pd.merge(SFUklass, skj45O, how="left", on=["ORGNR_FORETAK", "RHF", "NAVN_KLASS", "HELSEREGION"])
finne_virksomheter_df = finne_virksomheter_df.query('tmp_bool == True and SN07_1 == "86.105" and STATUS == "B"')
finne_virksomheter_df = finne_virksomheter_df[['ORGNR','ORGNR_FORETAK','NAVN']]

In [171]:
undervirksomheter_navn_og_kolonner = lag_navn_orgnr_kolonner(finne_virksomheter_df, 20)

In [172]:
skj45O = pd.merge(skj45O, undervirksomheter_navn_og_kolonner, how="left", on="ORGNR_FORETAK")

In [173]:
skj45O['USERID'] = skj45O['ORGNR_FORETAK']
skj45O['USERID'] = skj45O['ORGNR_FORETAK']
skj45O = skj45O.rename(columns={"ORGNR_FORETAK": "FORETAKETS_ORGNR",
                                "NAVN_KLASS": "FORETAKETS_NAVN",
                                "RHF": "HELSEREGION_NAVN"})

In [174]:
skj45O = pd.merge(skj45O, d_plass_fjor, how="left", on="FORETAKETS_ORGNR")

In [175]:
skj45O = skj45O[onskede_kolonner]

### skj46O (Somatiske sykehus, offentlige helseforetak)
```SUNNAAS``` skal være med her

NB. Antall avsatte kolonner til undervirksomheter er 24.

In [176]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata46O[['ORGNR_VIRK1','SEN_HT', 'SDGN_HT']].reset_index().copy()
                [['ORGNR_VIRK1','SEN_HT', 'SDGN_HT']]
                .rename(columns={'SEN_HT': 'SEN_HT_FJOR',
                                 'SDGN_HT': 'SDGN_HT_FJOR',
                                 'ORGNR_VIRK1': 'FINST_ORGNR'}))

In [177]:
skj46O = HF.copy()

In [178]:
# Lager kolonneoverskrifter i tråd med tidligere .csv-filer:
navn_virk, orgnr_virk = lag_navn_orgnr_kolonnenavn(24)

kolonner = ["USERID", "HELSEREGION", "HELSEREGION_NAVN",
            "FORETAKETS_ORGNR", "FORETAKETS_NAVN",
            'SEN_HT_FJOR', 'SDGN_HT_FJOR'] + orgnr_virk + navn_virk

Henter data fra SFU med næringskoder ("86.101") ("86.102") ("86.103") ("86.107"). Finner næringskoden i kolonne ```SN07_1```

In [179]:
skj46O['tmp_bool'] = True

finne_virksomheter_df = pd.merge(SFUklass, skj46O, how="left", on=["ORGNR_FORETAK", "RHF", "NAVN_KLASS", "HELSEREGION"])
finne_virksomheter_df = finne_virksomheter_df.query('tmp_bool == True and SN07_1 in ["86.101", "86.102", "86.103","86.107",]')

finne_virksomheter_df = finne_virksomheter_df[['ORGNR','ORGNR_FORETAK','NAVN', 'NAVN_KLASS']]

finne_virksomheter_df = pd.merge(finne_virksomheter_df, d_plass_fjor, how="left", left_on="ORGNR", right_on="FINST_ORGNR")

finne_virksomheter_df['SEN_HT_FJOR'] = pd.to_numeric(finne_virksomheter_df['SEN_HT_FJOR'])
finne_virksomheter_df['SDGN_HT_FJOR'] = pd.to_numeric(finne_virksomheter_df['SDGN_HT_FJOR'])

finne_virksomheter_df = finne_virksomheter_df.groupby(["ORGNR_FORETAK", "NAVN_KLASS"]).sum().reset_index()
finne_virksomheter_df = finne_virksomheter_df.rename(columns={"NAVN_KLASS": "FORETAKETS_NAVN"})

In [180]:
finne_virksomheter_df2 = pd.merge(SFUklass, skj46O, how="left", on=["ORGNR_FORETAK", "RHF", "NAVN_KLASS", "HELSEREGION"])
finne_virksomheter_df2 = finne_virksomheter_df2.query('tmp_bool == True and SN07_1 in ["86.101", "86.102", "86.103","86.107",]')

finne_virksomheter_df2 = finne_virksomheter_df2[['ORGNR','ORGNR_FORETAK','NAVN', 'NAVN_KLASS']]

In [181]:
undervirksomheter_navn_og_kolonner = lag_navn_orgnr_kolonner(finne_virksomheter_df2, 24, False)

skj46O = pd.merge(skj46O, undervirksomheter_navn_og_kolonner, how="left", on="ORGNR_FORETAK")

skj46O = pd.merge(skj46O, finne_virksomheter_df, how="left", on="ORGNR_FORETAK")

skj46O['USERID'] = skj46O['ORGNR_FORETAK']
skj46O = skj46O.rename(columns={"ORGNR_FORETAK": "FORETAKETS_ORGNR",
                                "NAVN_KLASS": "FORETAKETS_NAVN",
                                "RHF": "HELSEREGION_NAVN"})

In [182]:
skj46O = skj46O[kolonner]

## Skjemaer til private foretak og deres underinstitusjoner
Droplisten inneholder kolonne med rapporteringspliktige underinstitusjoner i en kolonne atskilt med \n

### Nyttige funksjoner som skal gjøre koden mer lesbar

In [183]:
# Henter foretaksnavn til virksomhetene fra SFU
def hent_foretaksnavn_til_virksomhetene_fra_SFU(unike_foretak):
    unike_foretak = pd.DataFrame(unike_foretak)
    unike_foretak.columns = ["ORGNR_FORETAK"]

    finn_foretaksnavn = pd.merge(unike_foretak, SFUklass[["ORGNR", "NAVN"]],
                                 how="left",
                                 left_on="ORGNR_FORETAK",
                                 right_on="ORGNR")
    finn_foretaksnavn = finn_foretaksnavn[["ORGNR_FORETAK", "NAVN"]]\
                                        .rename(columns={"NAVN": "FORETAK_NAVN",
                                        "ORGNR_FORETAK": "FORETAK_ORGNR"})
    return finn_foretaksnavn

tmp_kolonner = ["USERID", "REGION_NR", "REGION_NAVN", "FORETAK_ORGNR",
                "FORETAK_NAVN", "NAVN1", "FINST_NAVN","FINST_ORGNR"] + \
                ['H_VAR2_N'] + \
                ["SN07_1"]

### skj38P (TSB for private helseforetak)

Henter inn alle rader i SFU som har skjematype 381. Variabelen ```H_VAR1_A```  sier hvilket orgnummer foretaket rapporterer til. Koden under bruker denne til å variabelen til å plassere rapporteringsenhetene først, og alle deres underinstitusjoner i variabelen ```INSTLIST```. 

For at filen skal ha riktig format, skal det tilsammen være 13 institusjoner per foretak i ```INSTLIST```. Fyller automatisk ut linjeskift ```\n``` slik at antallet blir 13.

In [184]:
# kolonner som skal være med i droplisten:
kolonner = ['USERID',
            'REGION_NR',
            'REGION_NAVN',
            'FORETAK_ORGNR',
            'FORETAK_NAVN',
            'FINST_ORGNR',
            'FINST_NAVN',
            'INSTLIST']

# Kolonner med hjelpevariabler
hvar_kolonner = ["H_VAR1_N","H_VAR2_N","H_VAR3_N","H_VAR1_A","H_VAR2_A","H_VAR3_A"]

skj38P = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "381").copy()
skj38P = skj38P.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj38P['USERID'] = skj38P['FORETAK_ORGNR']


In [185]:
# Importerer riktig regionnummer fra KLASS og gir foretak som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj38P = pd.merge(skj38P, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj38P['REGION_NAVN'] = skj38P['RHF']
skj38P.REGION_NAVN = skj38P.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [186]:
foretaksnavn = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj38P.FORETAK_ORGNR.unique())
skj38P = pd.merge(skj38P, foretaksnavn, how="left", on="FORETAK_ORGNR")

In [187]:
skj38P = skj38P[tmp_kolonner]

In [188]:
# lager kolonnen INSTLIST med riktig antall \n for at det skal passe med skjema. 
# Passer på at rapporteringsenhet kommer først i listen. (Enhet som har H_VAR1_A lik orgnummer)

finst_orgnr38p = pd.DataFrame(skj38P['FINST_ORGNR'])

df_til_instlist1 = pd.merge(SFUklass, finst_orgnr38p, left_on="H_VAR1_A", right_on="FINST_ORGNR")
df_til_instlist1['rapporterer_til_annen_virksomhet'] = (df_til_instlist1.ORGNR != df_til_instlist1.H_VAR1_A)
df_til_instlist1 = df_til_instlist1[['NAVN', 'ORGNR', "FINST_ORGNR", 'rapporterer_til_annen_virksomhet']]
df_til_instlist1['orgnr_navn'] = df_til_instlist1['ORGNR'] + " - " + df_til_instlist1['NAVN']

df_til_instlist2 = df_til_instlist1.query('~rapporterer_til_annen_virksomhet')[['ORGNR', 'orgnr_navn']]
df_til_instlist3 = df_til_instlist1.query('rapporterer_til_annen_virksomhet')[["FINST_ORGNR", 'orgnr_navn']]


In [189]:
# mellomsteg for å legge institusjonene horisontalt ved siden av rapporteringsnummeret
# itererer over alle unike rapporteringsnummer
tmpdf = -1
for finst_orgnr in df_til_instlist2.ORGNR.unique():
    instliste = "\\n".join(df_til_instlist3.query(f'FINST_ORGNR=="{finst_orgnr}"').orgnr_navn)
    if len(instliste) > 0:
        if type(tmpdf) == int:
            tmpdf = pd.DataFrame([finst_orgnr, instliste])
        else:
            nytt_tillegg = pd.DataFrame([finst_orgnr, instliste])
            tmpdf = pd.concat([tmpdf, nytt_tillegg], axis=1)

tmpdf = tmpdf.transpose()
tmpdf.columns = ['FINST_ORGNR', 'INSTLISTE_HALE']

In [190]:
df_til_instlist2 = df_til_instlist2.rename(columns={'ORGNR': 'FINST_ORGNR'})

In [191]:
skj38P = pd.merge(skj38P, df_til_instlist2, how="left", on="FINST_ORGNR")
skj38P = pd.merge(skj38P, tmpdf, how="left", on="FINST_ORGNR")
skj38P['INSTLISTE_HALE'] = skj38P['INSTLISTE_HALE'].fillna("\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n")
skj38P['antall_n_mangler'] = skj38P['INSTLISTE_HALE'].apply(lambda x: 11-x.count("\\n"))
skj38P['INSTLIST'] = skj38P.apply(lambda x: x.orgnr_navn + \
                                            "\\n" + \
                                            x.INSTLISTE_HALE + \
                                            x.antall_n_mangler * ("\\n"),
                                  axis=1)

In [192]:
skj38P = skj38P[kolonner]

### skj39 (Resultatregnskap for private helseforetak)

Lages på nesten samme måte som skjema 38P, med unntak av at man henter ut SKJEMA_TYPE 39 fra SFU og legger til kolonnen ```INSTTYPE``` med institusjonstype hentet fra hjelpevariabelen ```SN07_1```.

In [193]:
# kolonner som skal være med i dropdownlisten:
kolonner = ['USERID',
            'REGION_NR',
            'REGION_NAVN',
            'FORETAK_ORGNR',
            'FORETAK_NAVN',
            'FINST_ORGNR',
            'FINST_NAVN',
            'INSTTYPE',
            'INSTLIST']

skj39 = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "39").copy()
skj39 = skj39.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj39['USERID'] = skj39['FORETAK_ORGNR']


In [194]:
# Importerer riktig regionnummer fra KLASS og gir foretak som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj39 = pd.merge(skj39, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj39['REGION_NAVN'] = skj39['RHF']
skj39.REGION_NAVN = skj39.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [195]:
# Henter foretaksnavn til virksomhetene fra SFU
foretak = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj39.FORETAK_ORGNR.unique())

skj39 = pd.merge(skj39, foretak, how="left", on="FORETAK_ORGNR")

In [196]:
skj39 = skj39[tmp_kolonner]


In [197]:
# lager kolonnen INSTLIST med riktig antall \n for at det skal passe med skjema. 
# Passer på at rapporteringsenhet kommer først i listen. (Enhet med H_VAR1_A lik orgnummer)

finst_orgnr39 = pd.DataFrame(skj39['FINST_ORGNR'])

df_til_instlist1 = pd.merge(SFUklass, finst_orgnr39, left_on="H_VAR1_A", right_on="FINST_ORGNR")
df_til_instlist1['rapporterer_til_annen_virksomhet'] = (df_til_instlist1.ORGNR != df_til_instlist1.H_VAR1_A)
df_til_instlist1 = df_til_instlist1[['NAVN', 'ORGNR', "FINST_ORGNR", 'rapporterer_til_annen_virksomhet']]
df_til_instlist1['orgnr_navn'] = df_til_instlist1['ORGNR'] + " - " + df_til_instlist1['NAVN']

df_til_instlist2 = df_til_instlist1.query('~rapporterer_til_annen_virksomhet')[['ORGNR', 'orgnr_navn']]
df_til_instlist3 = df_til_instlist1.query('rapporterer_til_annen_virksomhet')[["FINST_ORGNR", 'orgnr_navn']]


In [198]:
# mellomsteg for å legge institusjonene horisontalt ved siden av rapporteringsnummeret
tmpdf = -1
for finst_orgnr in df_til_instlist2.ORGNR.unique():
    instliste = "\\n".join(df_til_instlist3.query(f'FINST_ORGNR=="{finst_orgnr}"').orgnr_navn)
    if len(instliste) > 0:
        if type(tmpdf) == int:
            tmpdf = pd.DataFrame([finst_orgnr, instliste])
        else:
            nytt_tillegg = pd.DataFrame([finst_orgnr, instliste])
            tmpdf = pd.concat([tmpdf, nytt_tillegg], axis=1)

tmpdf = tmpdf.transpose()
tmpdf.columns = ['FINST_ORGNR', 'INSTLISTE_HALE']

In [199]:
df_til_instlist2 = df_til_instlist2.rename(columns={'ORGNR': 'FINST_ORGNR'})

In [200]:
skj39 = pd.merge(skj39, df_til_instlist2, how="left", on="FINST_ORGNR")
skj39 = pd.merge(skj39, tmpdf, how="left", on="FINST_ORGNR")
skj39['INSTLISTE_HALE'] = skj39['INSTLISTE_HALE'].fillna("\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n")
skj39['antall_n_mangler'] = skj39['INSTLISTE_HALE'].apply(lambda x: 11-x.count("\\n"))
skj39['INSTLIST'] = skj39.apply(lambda x: x.orgnr_navn + \
                                            "\\n" + \
                                            x.INSTLISTE_HALE + \
                                            x.antall_n_mangler * ("\\n"),
                                  axis=1)

In [201]:
institusjonstype = {
    451: 'Psykisk helsevern for barn og unge',
    461: 'Somatiske sykehus',
    381: 'Rusmiddelinstitusjoner',
    441: 'Psykisk helsevern for voksne',
    47:  'Somatiske rehab.-og opptr.inst.'
}

skj39 = skj39.rename(columns={'H_VAR2_N': 'INSTTYPE'})

In [202]:
skj39['INSTTYPE'] = skj39['INSTTYPE'].map(institusjonstype)

In [203]:
# Hvis FORETAK_NAVN er tom, bruk NAVN1 fra SFU:
skj39.loc[skj39['FORETAK_NAVN'].isnull(),'FORETAK_NAVN'] = skj39['NAVN1']

In [204]:
# Tar kun vare på de kolonnene jeg spesifiserer i begynnelsen
skj39 = skj39[kolonner]

### skj44P (Psykisk helsevern for voksne (PHFV), private helseforetak)

In [205]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata44P[['FINST_ORGNR','D_PLAS_T']].reset_index().copy()
                [['FINST_ORGNR','D_PLAS_T']]
                .rename(columns={'D_PLAS_T': 'D_PLAS_FJOR'}))

In [206]:
# kolonner som skal være med i dropdownlisten:
kolonner = ['USERID',
            'REGION_NR',
            'REGION_NAVN',
            'FORETAK_ORGNR',
            'FORETAK_NAVN',
            'FINST_ORGNR',
            'FINST_NAVN',
            'INSTLIST',
            'D_PLAS_FJOR']

skj44P = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "441").copy()
skj44P = skj44P.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj44P['USERID'] = skj44P['FORETAK_ORGNR']

# Lager omrisset av endelig tabell ved å plukke ut relevante kolonner
skj44P = skj44P[["USERID", "REGION_NR", "REGION_NAVN", "FORETAK_ORGNR", "FINST_NAVN", "FINST_ORGNR"] +
                hvar_kolonner +
                ["SN07_1", "SN07_2", "SN07_3"]]


In [207]:
# Importerer riktig regionnummer fra KLASS og gir foretak som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj44P = pd.merge(skj44P, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj44P['REGION_NAVN'] = skj44P['RHF']
skj44P.REGION_NAVN = skj44P.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [208]:
# Henter foretaksnavn til virksomhetene fra SFU
foretak = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj44P.FORETAK_ORGNR.unique())

skj44P = pd.merge(skj44P, foretak, how="left", on="FORETAK_ORGNR")

In [209]:
skj44P = skj44P[tmp_kolonner]


KeyError: "['NAVN1'] not in index"

In [ ]:
# lager kolonnen INSTLIST med riktig antall \n for at det skal passe med skjema. 
# Passer på at rapporteringsenhet kommer først i listen. (Enhet med H_VAR1_A lik orgnummer)

finst_orgnr44P = pd.DataFrame(skj44P['FINST_ORGNR'])

df_til_instlist1 = pd.merge(SFUklass, finst_orgnr44P, left_on="H_VAR1_A", right_on="FINST_ORGNR")
df_til_instlist1['rapporterer_til_annen_virksomhet'] = (df_til_instlist1.ORGNR != df_til_instlist1.H_VAR1_A)
df_til_instlist1 = df_til_instlist1[['NAVN', 'ORGNR', "FINST_ORGNR", 'rapporterer_til_annen_virksomhet']]
df_til_instlist1['orgnr_navn'] = df_til_instlist1['ORGNR'] + " - " + df_til_instlist1['NAVN']

df_til_instlist2 = df_til_instlist1.query('~rapporterer_til_annen_virksomhet')[['ORGNR', 'orgnr_navn']]
df_til_instlist3 = df_til_instlist1.query('rapporterer_til_annen_virksomhet')[["FINST_ORGNR", 'orgnr_navn']]


In [ ]:
# mellomsteg for å legge institusjonene horisontalt ved siden av rapporteringsnummeret
tmpdf = -1
for finst_orgnr in df_til_instlist2.ORGNR.unique():
    instliste = "\\n".join(df_til_instlist3.query(f'FINST_ORGNR=="{finst_orgnr}"').orgnr_navn)
    if len(instliste) > 0:
        if type(tmpdf) == int:
            tmpdf = pd.DataFrame([finst_orgnr, instliste])
        else:
            nytt_tillegg = pd.DataFrame([finst_orgnr, instliste])
            tmpdf = pd.concat([tmpdf, nytt_tillegg], axis=1)

tmpdf = tmpdf.transpose()
tmpdf.columns = ['FINST_ORGNR', 'INSTLISTE_HALE']

In [ ]:
df_til_instlist2 = df_til_instlist2.rename(columns={'ORGNR': 'FINST_ORGNR'})

In [ ]:
skj44P = pd.merge(skj44P, df_til_instlist2, how="left", on="FINST_ORGNR")
skj44P = pd.merge(skj44P, tmpdf, how="left", on="FINST_ORGNR")
skj44P['INSTLISTE_HALE'] = skj44P['INSTLISTE_HALE'].fillna("\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n")
skj44P['antall_n_mangler'] = skj44P['INSTLISTE_HALE'].apply(lambda x: 11-x.count("\\n"))
skj44P['INSTLIST'] = skj44P.apply(lambda x: x.orgnr_navn + \
                                            "\\n" + \
                                            x.INSTLISTE_HALE + \
                                            x.antall_n_mangler * ("\\n"),
                                  axis=1)

In [ ]:
skj44P = pd.merge(skj44P, d_plass_fjor, how="left", on="FINST_ORGNR")

In [ ]:
# Tar kun vare på de kolonnene jeg spesifiserte i begynnelsen 
skj44P = skj44P[kolonner]

### skj45P (Psykisk helsevern for barn og unge (PHBU/BUP), private helseforetak)

In [ ]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata45P[['FINST_ORGNR','D_PLAS_T']].reset_index().copy()
                [['FINST_ORGNR','D_PLAS_T']]
                .rename(columns={'D_PLAS_T': 'D_PLAS_FJOR'}))

In [ ]:
# kolonner som skal være med i dropdownlisten:
kolonner = "USERID	REGION_NR	REGION_NAVN	FORETAK_ORGNR	FORETAK_NAVN	FINST_ORGNR	FINST_NAVN	INSTLIST	D_PLAS_FJOR".split("	")

skj45P = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "451").copy()
skj45P = skj45P.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj45P['USERID'] = skj45P['FORETAK_ORGNR']

# Lager omrisset av endelig tabell ved å plukke ut relevante kolonner
skj45P = skj45P[["USERID", "REGION_NR", "REGION_NAVN", "FORETAK_ORGNR", "FINST_NAVN", "FINST_ORGNR"] +
                hvar_kolonner +
                ["SN07_1", "SN07_2", "SN07_3"]]


In [ ]:
# Importerer riktig regionnummer fra KLASS og gir foretak som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj45P = pd.merge(skj45P, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj45P['REGION_NAVN'] = skj45P['RHF']
skj45P.REGION_NAVN = skj45P.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [ ]:
# Henter foretaksnavn til virksomhetene fra SFU
foretak = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj45P.FORETAK_ORGNR.unique())

skj45P = pd.merge(skj45P, foretak, how="left", on="FORETAK_ORGNR")

In [ ]:
skj45P = skj45P[tmp_kolonner]


In [ ]:
# lager kolonnen INSTLIST med riktig antall \n for at det skal passe med skjema. 
# Passer på at rapporteringsenhet kommer først i listen. (Enhet med H_VAR1_A lik orgnummer)

finst_orgnr45P = pd.DataFrame(skj45P['FINST_ORGNR'])

df_til_instlist1 = pd.merge(SFUklass, finst_orgnr45P, left_on="H_VAR1_A", right_on="FINST_ORGNR")
df_til_instlist1['rapporterer_til_annen_virksomhet'] = (df_til_instlist1.ORGNR != df_til_instlist1.H_VAR1_A)
df_til_instlist1 = df_til_instlist1[['NAVN', 'ORGNR', "FINST_ORGNR", 'rapporterer_til_annen_virksomhet']]
df_til_instlist1['orgnr_navn'] = df_til_instlist1['ORGNR'] + " - " + df_til_instlist1['NAVN']

df_til_instlist2 = df_til_instlist1.query('~rapporterer_til_annen_virksomhet')[['ORGNR', 'orgnr_navn']]
df_til_instlist3 = df_til_instlist1.query('rapporterer_til_annen_virksomhet')[["FINST_ORGNR", 'orgnr_navn']]


In [ ]:
# mellomsteg for å legge institusjonene horisontalt ved siden av rapporteringsnummeret
tmpdf = -1
for finst_orgnr in df_til_instlist2.ORGNR.unique():
    instliste = "\\n".join(df_til_instlist3.query(f'FINST_ORGNR=="{finst_orgnr}"').orgnr_navn)
    if len(instliste) > 0:
        if type(tmpdf) == int:
            tmpdf = pd.DataFrame([finst_orgnr, instliste])
        else:
            nytt_tillegg = pd.DataFrame([finst_orgnr, instliste])
            tmpdf = pd.concat([tmpdf, nytt_tillegg], axis=1)

if type(tmpdf) != int:
    tmpdf = tmpdf.transpose()
    tmpdf.columns = ['FINST_ORGNR', 'INSTLISTE_HALE']

In [ ]:
df_til_instlist2 = df_til_instlist2.rename(columns={'ORGNR': 'FINST_ORGNR'})

In [ ]:
skj45P = pd.merge(skj45P, df_til_instlist2, how="left", on="FINST_ORGNR")
if type(tmpdf) != int:
    skj45P = pd.merge(skj45P, tmpdf, how="left", on="FINST_ORGNR")
    skj45P['INSTLISTE_HALE'] = skj45P['INSTLISTE_HALE'].fillna("\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n")
else:
    skj45P['INSTLISTE_HALE'] = "\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n"

skj45P['antall_n_mangler'] = skj45P['INSTLISTE_HALE'].apply(lambda x: 11-x.count("\\n"))
skj45P['INSTLIST'] = skj45P.apply(lambda x: x.orgnr_navn + \
                                            "\\n" + \
                                            x.INSTLISTE_HALE + \
                                            x.antall_n_mangler * ("\\n"),
                                  axis=1)

In [ ]:
skj45P = pd.merge(skj45P, d_plass_fjor, how="left", on="FINST_ORGNR")

In [ ]:
# Tar kun vare på de kolonnene jeg spesifiserte i begynnelsen 
skj45P = skj45P[kolonner]

### skj46P (Somatiske sykehus, private helseforetak)

In [ ]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata46P[['FINST_ORGNR','SEN_HT', 'SDGN_HT']].reset_index().copy()
                [['FINST_ORGNR','SEN_HT', 'SDGN_HT']]
                .rename(columns={'SEN_HT': 'SEN_HT_FJOR',
                                 'SDGN_HT': 'SDGN_HT_FJOR'}))

In [ ]:
# kolonner som skal være med i dropdownlisten:
kolonner = ['USERID',
            'REGION_NR',
            'REGION_NAVN',
            'FORETAK_ORGNR',
            'FORETAK_NAVN',
            'FINST_ORGNR',
            'FINST_NAVN',
            'INSTLIST',
            'SEN_HT_FJOR',
            'SDGN_HT_FJOR']

skj46P = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "461").copy()
skj46P = skj46P.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj46P['USERID'] = skj46P['FORETAK_ORGNR']

# Lager omrisset av endelig tabell ved å plukke ut relevante kolonner
skj46P = skj46P[["USERID", "REGION_NR", "REGION_NAVN", "FORETAK_ORGNR", "FINST_NAVN", "FINST_ORGNR", "NAVN1"] +
                hvar_kolonner +
                ["SN07_1", "SN07_2", "SN07_3"]]


In [ ]:
# Importerer riktig regionnummer fra KLASS og gir foretak som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj46P = pd.merge(skj46P, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj46P['REGION_NAVN'] = skj46P['RHF']
skj46P.REGION_NAVN = skj46P.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [ ]:
# Henter foretaksnavn til virksomhetene fra SFU
foretak = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj46P.FORETAK_ORGNR.unique())

skj46P = pd.merge(skj46P, foretak, how="left", on="FORETAK_ORGNR")

In [ ]:
skj46P = skj46P[tmp_kolonner]


In [ ]:
# lager kolonnen INSTLIST med riktig antall \n for at det skal passe med skjema. 
# Passer på at rapporteringsenhet kommer først i listen. (Enhet med H_VAR1_A lik orgnummer)

finst_orgnr46P = pd.DataFrame(skj46P['FINST_ORGNR'])

df_til_instlist1 = pd.merge(SFUklass, finst_orgnr46P, left_on="H_VAR1_A", right_on="FINST_ORGNR")
df_til_instlist1['rapporterer_til_annen_virksomhet'] = (df_til_instlist1.ORGNR != df_til_instlist1.H_VAR1_A)
df_til_instlist1 = df_til_instlist1[['NAVN', 'ORGNR', "FINST_ORGNR", 'rapporterer_til_annen_virksomhet']]
df_til_instlist1['orgnr_navn'] = df_til_instlist1['ORGNR'] + " - " + df_til_instlist1['NAVN']

df_til_instlist2 = df_til_instlist1.query('~rapporterer_til_annen_virksomhet')[['ORGNR', 'orgnr_navn']]
df_til_instlist3 = df_til_instlist1.query('rapporterer_til_annen_virksomhet')[["FINST_ORGNR", 'orgnr_navn']]


In [ ]:
# mellomsteg for å legge institusjonene horisontalt ved siden av rapporteringsnummeret
tmpdf = -1
for finst_orgnr in df_til_instlist2.ORGNR.unique():
    instliste = "\\n".join(df_til_instlist3.query(f'FINST_ORGNR=="{finst_orgnr}"').orgnr_navn)
    if len(instliste) > 0:
        if type(tmpdf) == int:
            tmpdf = pd.DataFrame([finst_orgnr, instliste])
        else:
            nytt_tillegg = pd.DataFrame([finst_orgnr, instliste])
            tmpdf = pd.concat([tmpdf, nytt_tillegg], axis=1)

tmpdf = tmpdf.transpose()
tmpdf.columns = ['FINST_ORGNR', 'INSTLISTE_HALE']

In [ ]:
df_til_instlist2 = df_til_instlist2.rename(columns={'ORGNR': 'FINST_ORGNR'})

In [ ]:
skj46P = pd.merge(skj46P, df_til_instlist2, how="left", on="FINST_ORGNR")
skj46P = pd.merge(skj46P, tmpdf, how="left", on="FINST_ORGNR")
skj46P['INSTLISTE_HALE'] = skj46P['INSTLISTE_HALE'].fillna("\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n")
skj46P['antall_n_mangler'] = skj46P['INSTLISTE_HALE'].apply(lambda x: 11-x.count("\\n"))
skj46P['INSTLIST'] = skj46P.apply(lambda x: x.orgnr_navn + \
                                            "\\n" + \
                                            x.INSTLISTE_HALE + \
                                            x.antall_n_mangler * ("\\n"),
                                  axis=1)

In [ ]:
skj46P = pd.merge(skj46P, d_plass_fjor, how="left", on="FINST_ORGNR")

In [ ]:
skj46P.loc[skj46P.FORETAK_NAVN.isnull(), 'FORETAK_NAVN'] = skj46P.NAVN1

In [ ]:
# Tar kun vare på de kolonnene jeg spesifiserte i begynnelsen 
skj46P = skj46P[kolonner]

### skj47 (Somatiske institusjoner)

In [ ]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata47[['FINST_ORGNR','D_HELD', 'SDGN_SUM']].reset_index().copy()
                [['FINST_ORGNR','D_HELD', 'SDGN_SUM']]
                .rename(columns={'D_HELD': 'D_HELD_IFJOR',
                                 'SDGN_SUM': 'SDGN_SUM_IFJOR'}))

In [ ]:
# kolonner som skal være med i dropdownlisten:
kolonner = ['USERID',
            'REGION_NR',
            'REGION_NAVN',
            'FORETAK_ORGNR',
            'FORETAK_NAVN',
            'FINST_ORGNR',
            'FINST_NAVN',
            'INSTLIST',
            'D_HELD_IFJOR',
            'SDGN_SUM_IFJOR']

skj47 = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "47").copy()
skj47 = skj47.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj47['USERID'] = skj47['FORETAK_ORGNR']

# Lager omrisset av endelig tabell ved å plukke ut relevante kolonner
skj47 = skj47[["USERID", "REGION_NR", "REGION_NAVN", "FORETAK_ORGNR", "FINST_NAVN", "NAVN1", "FINST_ORGNR"] +
                hvar_kolonner +
                ["SN07_1", "SN07_2", "SN07_3"]]


In [ ]:
# Importerer riktig regionnummer fra KLASS og gir foretak som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj47 = pd.merge(skj47, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj47['REGION_NAVN'] = skj47['RHF']
skj47.REGION_NAVN = skj47.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [ ]:
# Henter foretaksnavn til virksomhetene fra SFU
foretak = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj47.FORETAK_ORGNR.unique())
skj47 = pd.merge(skj47, foretak, how="left", on="FORETAK_ORGNR")

In [ ]:
skj47 = skj47[tmp_kolonner]

In [ ]:
# lager kolonnen INSTLIST med riktig antall \n for at det skal passe med skjema. 
# Passer på at rapporteringsenhet kommer først i listen. (Enhet med H_VAR1_A lik orgnummer)

finst_orgnr47 = pd.DataFrame(skj47['FINST_ORGNR'])

df_til_instlist1 = pd.merge(SFUklass, finst_orgnr47, left_on="H_VAR1_A", right_on="FINST_ORGNR")
df_til_instlist1['rapporterer_til_annen_virksomhet'] = (df_til_instlist1.ORGNR != df_til_instlist1.H_VAR1_A)
df_til_instlist1 = df_til_instlist1[['NAVN', 'ORGNR', "FINST_ORGNR", 'rapporterer_til_annen_virksomhet']]
df_til_instlist1['orgnr_navn'] = df_til_instlist1['ORGNR'] + " - " + df_til_instlist1['NAVN']

df_til_instlist2 = df_til_instlist1.query('~rapporterer_til_annen_virksomhet')[['ORGNR', 'orgnr_navn']]
df_til_instlist3 = df_til_instlist1.query('rapporterer_til_annen_virksomhet')[["FINST_ORGNR", 'orgnr_navn']]


In [ ]:
# mellomsteg for å legge institusjonene horisontalt ved siden av rapporteringsnummeret
tmpdf = -1
for finst_orgnr in df_til_instlist2.ORGNR.unique():
    instliste = "\\n".join(df_til_instlist3.query(f'FINST_ORGNR=="{finst_orgnr}"').orgnr_navn)
    if len(instliste) > 0:
        if type(tmpdf) == int:
            tmpdf = pd.DataFrame([finst_orgnr, instliste])
        else:
            nytt_tillegg = pd.DataFrame([finst_orgnr, instliste])
            tmpdf = pd.concat([tmpdf, nytt_tillegg], axis=1)

tmpdf = tmpdf.transpose()
tmpdf.columns = ['FINST_ORGNR', 'INSTLISTE_HALE']

In [ ]:
df_til_instlist2 = df_til_instlist2.rename(columns={'ORGNR': 'FINST_ORGNR'})

In [ ]:
skj47 = pd.merge(skj47, df_til_instlist2, how="left", on="FINST_ORGNR")
skj47 = pd.merge(skj47, tmpdf, how="left", on="FINST_ORGNR")
skj47['INSTLISTE_HALE'] = skj47['INSTLISTE_HALE'].fillna("\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n")
skj47['antall_n_mangler'] = skj47['INSTLISTE_HALE'].apply(lambda x: 11-x.count("\\n"))
skj47['INSTLIST'] = skj47.apply(lambda x: x.orgnr_navn + \
                                            "\\n" + \
                                            x.INSTLISTE_HALE + \
                                            x.antall_n_mangler * ("\\n"),
                                  axis=1)

In [ ]:
skj47 = pd.merge(skj47, d_plass_fjor, how="left", on="FINST_ORGNR")

In [ ]:
# Hvis FORETAK_NAVN er tom, bruk NAVN1 fra SFU:
skj47.loc[skj47['FORETAK_NAVN'].isnull(),'FORETAK_NAVN'] = skj47['NAVN1']

In [ ]:
# Tar kun vare på de kolonnene jeg spesifiserte i begynnelsen
skj47 = skj47[kolonner]

# Eksportering til .csv-filer og rapport

In [ ]:
def rapport(skj):
    rader = eval(skj).shape[0]
    rader_med_missing = rader - eval(skj).dropna().shape[0]
    skl = 40 * "-"
    s = f"{skl}\n{skj}\nRader: {rader} \t Rader med missing: {rader_med_missing}"
    return s

def lagre_dropliste_csv(skj):
    filnavn = "Dropliste_" + skj + "_" + str(aargang) + ".csv"
    eval(skj).to_csv(sti_til_lagring + filnavn, sep=";", encoding='latin1', index=False)

In [ ]:
for x in skjemaer_til_droplister:
    print(rapport(x))

In [ ]:
if lagre_filer == True:
    for x in skjemaer_til_droplister:
        lagre_dropliste_csv(x)